In [3]:
import pandas as pd
import sklearn

In [4]:
df = pd.read_csv('LA o3.csv', parse_dates=['date','utc','local'], index_col='utc')

In [5]:
df.head()

,parameter,location,value,concentration_units,city,country,sourcename,sourcetype,mobile,latitude,...,stp,visibility,wind_speed,max_sustained_wind,gust,max_temp,min_temp,precipitation,snow_depth,weather_events
utc,,,,,,,,,,,,,,,,,,,,,
2020-01-02 21:00:00,o3,North Holywood,0.036,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN
2020-01-02 15:00:00,o3,North Holywood,0.018,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN
2020-01-02 20:00:00,o3,North Holywood,0.041,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN
2020-01-02 23:00:00,o3,North Holywood,0.031,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN
2020-01-02 17:00:00,o3,North Holywood,0.034,ppm,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN


In [6]:
df.columns

Index(['parameter', 'location', 'value', 'concentration_units', 'city',
       'country', 'sourcename', 'sourcetype', 'mobile', 'latitude',
       'longitude', 'local', 'source', 'url', 'averaging_time_unit',
       'averaging_time', 'station', 'date', 'latitude-2', 'longitude-2',
       'elevation', 'name', 'temperature', 'dewpoint', 'slp', 'stp',
       'visibility', 'wind_speed', 'max_sustained_wind', 'gust', 'max_temp',
       'min_temp', 'precipitation', 'snow_depth', 'weather_events'],
      dtype='object')

In [7]:
df = df.resample('d', on='date').mean()
df = df.interpolate(method='time')
df.head()

,value,mobile,latitude,longitude,url,averaging_time,station,latitude-2,longitude-2,elevation,...,slp,stp,visibility,wind_speed,max_sustained_wind,gust,max_temp,min_temp,precipitation,snow_depth
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,0.032111,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1012.5,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0.0
2020-01-03,0.008792,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1019.9,994.0,10.0,1.6,6.0,NaN,73.0,45.0,0.0,0.0
2020-01-04,0.009125,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1025.1,999.1,10.0,1.1,6.0,NaN,73.0,42.1,0.0,0.0
2020-01-05,0.012125,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1024.2,998.0,10.0,2.8,11.1,18.100000,73.0,42.1,0.0,0.0
2020-01-06,0.020143,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1023.5,997.6,10.0,2.5,6.0,16.733333,75.0,42.1,0.0,0.0


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 123)
X = train_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_test = test_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

lrmodel = LinearRegression()
lrmodel.fit(X,y)

y_pred = lrmodel.predict(X)
print(f'Results for linear regression on training data')
print(f'    Default Settings')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
    Default Settings
Internal Parameters:
   Bias is 0.3576625981042214
   Coefficients: [-3.70170889e-04  1.96755682e-03  9.08430012e-05 -3.62439163e-04
 -3.78688765e-06  8.18600528e-04  1.10611164e-04]
   Score 0.45130104711617447
MAE is 0.006572341986743905
RMSE is 0.00869120013859023
MSE is 7.553695984903081e-05
R^2: 0.45130104711617447


In [9]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor

power = 8
poly_process = PolynomialFeatures(degree = power, include_bias=True)

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 123)
X = train_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_poly = poly_process.fit_transform(X)

X_test = test_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

X_poly_test = poly_process.fit_transform(X_test)

lrmodel = LinearRegression()
lrmodel.fit(X_poly ,y)

y_pred = lrmodel.predict(X_poly)
print(f'Results for linear regression on training data')
print(f'Input: Height, age')
print(f'    Default Settings')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X_poly,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Input: Height, age
    Default Settings
Internal Parameters:
   Bias is -3134.4650031896917
   Coefficients: [-1.90560245e-12 -2.09002183e-12  4.26269093e-13 ...  1.13164622e-12
 -1.05150496e-12 -3.65254301e-12]
   Score -0.07017577429194666
MAE is 0.00910037317818601
RMSE is 0.012137807428319873
MSE is 0.00014732636916697708
R^2: -0.07017577429194666


In [10]:
y_test_pred = lrmodel.predict(X_poly_test)
print()
print(f'Results for linear regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
    Polynomial regression with degree 8
MAE is 1.7500184589951
RMSE is 8.384902598331532
MSE is 70.30659158350689
R^2: -468808.57342595316


In [11]:
power = 2
poly_process = PolynomialFeatures(degree = power, include_bias=True)

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 123)
X = train_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_poly = poly_process.fit_transform(X)

X_test = test_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

X_poly_test = poly_process.fit_transform(X_test)

lrmodel = LinearRegression()
lrmodel.fit(X_poly ,y)

y_pred = lrmodel.predict(X_poly)
print(f'Results for linear regression on training data')
print(f'Input: Height, age')
print(f'    Default Settings')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X_poly,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Input: Height, age
    Default Settings
Internal Parameters:
   Bias is 8.279253898987445
   Coefficients: [ 1.31602836e-12 -1.59422070e-02  9.65780027e-02 -1.05307194e-01
 -2.70660481e-02 -2.15402061e-02  5.51991354e-02 -3.10913271e-03
  7.64895752e-06 -8.85927727e-05  1.04566602e-04  2.45166808e-05
  2.48149912e-05 -5.42628467e-05  2.66644168e-06 -4.28822454e-04
  2.72396484e-04  3.53799479e-05  1.77231699e-04 -2.12994617e-04
 -9.89187485e-05 -7.01030415e-05  3.61376278e-05 -9.69020494e-05
  3.39420265e-05  5.83195983e-05 -4.11432800e-05 -4.31461504e-05
  1.50016347e-04  2.69038306e-05 -2.96360915e-05  4.60874004e-05
  2.23064996e-05 -9.82164508e-05 -4.69518280e-05  1.22004365e-06]
   Score 0.5292414163239798
MAE is 0.005986997988129689
RMSE is 0.00805029525423873
MSE is 6.480725368041862e-05
R^2: 0.5292414163239798


In [12]:
y_test_pred = lrmodel.predict(X_poly_test)
print()
print(f'Results for linear regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
    Polynomial regression with degree 2
MAE is 0.006445862912366921
RMSE is 0.008884584726659674
MSE is 7.893584576519435e-05
R^2: 0.47364991890918595


In [13]:
power = 3
poly_process = PolynomialFeatures(degree = power, include_bias=True)

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 123)
X = train_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_poly = poly_process.fit_transform(X)

X_test = test_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

X_poly_test = poly_process.fit_transform(X_test)

lrmodel = LinearRegression()
lrmodel.fit(X_poly ,y)

y_pred = lrmodel.predict(X_poly)
print(f'Results for linear regression on training data')
print(f'Input: Height, age')
print(f'    Default Settings')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X_poly,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Input: Height, age
    Default Settings
Internal Parameters:
   Bias is -1537.5978379885937
   Coefficients: [-3.11633207e-05  4.60831347e+00  4.69508396e+00 -9.95504759e+00
  1.43238227e+01  1.14897323e+01 -2.15296603e+01 -4.93216788e+00
 -4.60562374e-03 -6.59090227e-03  1.89991036e-02 -2.78146664e-02
 -2.29799455e-02  4.20579958e-02  9.94329832e-03 -5.56093774e-02
 -3.09729827e-02  2.27161066e-02 -2.42876236e-02 -4.98533897e-02
  2.15561098e-02  2.93184035e-02 -2.63991233e-02  1.43148647e-02
  3.11877509e-02 -6.75949486e-03  2.33531567e-02  2.65286495e-02
 -9.68763562e-02  2.36944882e-03  1.12902617e-02 -5.43912998e-02
 -2.42605110e-03  9.37322470e-02 -2.27163517e-03  1.02224291e-03
  1.53501405e-06  1.91293209e-06 -9.05242144e-06  1.35001869e-05
  1.14900606e-05 -2.05433265e-05 -5.01191232e-06  5.99422625e-05
  2.59125280e-05 -2.26308732e-05  2.46733962e-05  4.92053703e-05
 -2.21120252e-05 -2.86049338e-05  2.59833614e-05 -1.44781182e-05

In [14]:
y_test_pred = lrmodel.predict(X_poly_test)
print()
print(f'Results for linear regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
    Polynomial regression with degree 3
MAE is 0.006890540947847141
RMSE is 0.009434882429815746
MSE is 8.901700646444586e-05
R^2: 0.40642799077118075


Degree 3 is overfit

In [15]:
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators=1000, max_features = 'sqrt', max_depth =100,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.002224936615112716
RMSE is 0.003000061024320119
MSE is 9.000366149644682e-06
R^2: 0.9346215218119553


In [16]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for random forest regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for random forest regression on test data
    Polynomial regression with degree 3
MAE is 0.006176339060595092
RMSE is 0.008766595902347633
MSE is 7.68532037150583e-05
R^2: 0.4875371307499642


In [17]:
rfmodel = RandomForestRegressor(n_estimators=100, max_features = 'sqrt', max_depth =100,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.002264266917547745
RMSE is 0.0030790088082330133
MSE is 9.48029524117648e-06
R^2: 0.9311353265704697


In [18]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for random forest regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for random forest regression on test data
    Polynomial regression with degree 3
MAE is 0.006192921445541536
RMSE is 0.008806633637455421
MSE is 7.755679602436131e-05
R^2: 0.48284552498497213


In [19]:
rfmodel = RandomForestRegressor(n_estimators=100, max_features = 'sqrt', max_depth =50,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.002264266917547745
RMSE is 0.0030790088082330133
MSE is 9.48029524117648e-06
R^2: 0.9311353265704697


In [20]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for random forest regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for random forest regression on test data
    Polynomial regression with degree 3
MAE is 0.006192921445541536
RMSE is 0.008806633637455421
MSE is 7.755679602436131e-05
R^2: 0.48284552498497213


In [21]:
rfmodel = RandomForestRegressor(n_estimators=100, max_features = 'sqrt', max_depth =15,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.0025036107163972643
RMSE is 0.003346977306593721
MSE is 1.1202257090853359e-05
R^2: 0.9186270304236303


In [22]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for random forest regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for random forest regression on test data
    Polynomial regression with degree 3
MAE is 0.006123570360884102
RMSE is 0.008695037790951965
MSE is 7.560368218608282e-05
R^2: 0.49586903309072505


In [23]:
rfmodel = RandomForestRegressor(n_estimators=100, max_features = 'sqrt', max_depth =10,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.0034786996890069717
RMSE is 0.004587066980801252
MSE is 2.104118348635711e-05
R^2: 0.8471572675220829


In [24]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for random forest regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for random forest regression on test data
    Polynomial regression with degree 3
MAE is 0.006219390282002631
RMSE is 0.008695311010704677
MSE is 7.5608433572882e-05
R^2: 0.4958373504907251


In [25]:
rfmodel = RandomForestRegressor(n_estimators=100, max_features = 'sqrt', max_depth =5,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.005394574729607386
RMSE is 0.007193499169853617
MSE is 5.174643030668468e-05
R^2: 0.624114973894888


In [26]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for random forest regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for random forest regression on test data
    Polynomial regression with degree 3
MAE is 0.0061860401329385615
RMSE is 0.008674271502926942
MSE is 7.524298610649045e-05
R^2: 0.49827418133360735


In [27]:
rfmodel = RandomForestRegressor(n_estimators=100, max_features = 'sqrt', max_depth =3,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.006096161166400397
RMSE is 0.00814075268095928
MSE is 6.627185421254569e-05
R^2: 0.5186025875972566


In [28]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for random forest regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for random forest regression on test data
    Polynomial regression with degree 3
MAE is 0.006246063929193036
RMSE is 0.00883092358465195
MSE is 7.798521135796202e-05
R^2: 0.47998881972774265


In [29]:
rfmodel = RandomForestRegressor(n_estimators=1000, max_features = 'sqrt', max_depth =3,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.0060665317907326495
RMSE is 0.008099332184213047
MSE is 6.559918183022929e-05
R^2: 0.5234888662156779


In [30]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for random forest regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for random forest regression on test data
    Polynomial regression with degree 3
MAE is 0.006236297337698707
RMSE is 0.008818417232441322
MSE is 7.776448248541807e-05
R^2: 0.48146065624049994


In [31]:
rfmodel = RandomForestRegressor(n_estimators=100000, max_features = 'sqrt', max_depth =3,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.006049384361552097
RMSE is 0.008085472359325795
MSE is 6.537486327342143e-05
R^2: 0.5251183116881841


In [32]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

mlpmodel = MLPRegressor(hidden_layer_sizes=(100,400,100), activation='tanh', random_state =123, max_iter=2000)
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X)
X_testscaled=sc_X.transform(X_test)
mlpmodel.fit(X_trainscaled,y)
y_pred = mlpmodel.predict(X_trainscaled)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.008900019705919597
RMSE is 0.012082576088924506
MSE is 0.00014598864494465023
R^2: -0.060458572520720066


In [33]:
y_test_pred = mlpmodel.predict(X_testscaled)
print()
print(f'Results for linear regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
MAE is 0.009196802676359564
RMSE is 0.012175513358541097
MSE is 0.00014824312554401268
R^2: 0.011503830802658599


In [34]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

mlpmodel = MLPRegressor(hidden_layer_sizes=(25,50,25), activation='tanh', random_state =123, max_iter=2000)
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X)
X_testscaled=sc_X.transform(X_test)
mlpmodel.fit(X_trainscaled,y)
y_pred = mlpmodel.predict(X_trainscaled)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 0.020642612512759586
RMSE is 0.026736781780683182
MSE is 0.0007148554999878721
R^2: -4.1926959344210575


In [35]:
y_test_pred = mlpmodel.predict(X_testscaled)
print()
print(f'Results for MLP Regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for MLP Regression on test data
MAE is 0.020703463248651202
RMSE is 0.027836104386024636
MSE is 0.00077484870738966
R^2: -4.166748718710657


In [36]:
X = train_set[['stp', 'wind_speed', 'max_temp' , 'dewpoint']]
y = train_set['value']

X_test = test_set[['stp', 'wind_speed', 'max_temp' , 'dewpoint']]
y_test = test_set['value']

lrmodel = LinearRegression()
lrmodel.fit(X,y)

y_pred = lrmodel.predict(X)
print(f'Results for linear regression on training data')
print(f'Input: Height, age')
print(f'    Default Settings')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Input: Height, age
    Default Settings
Internal Parameters:
   Bias is 0.4230642781356687
   Coefficients: [-0.00043557  0.00243388  0.00025176  0.00016548]
   Score 0.41983889473251734
MAE is 0.006783347557548184
RMSE is 0.008936901885977625
MSE is 7.986821531959042e-05
R^2: 0.41983889473251734


In [18]:
X = train_set[['wind_speed']]
y = train_set['value']

X_test = test_set[['wind_speed']]
y_test = test_set['value']

lrmodel = LinearRegression()
lrmodel.fit(X,y)

y_pred = lrmodel.predict(X)
print(f'Results for linear regression on training data')
print(f'    Default Settings')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
    Default Settings
Internal Parameters:
   Bias is 0.01597390578401982
   Coefficients: [0.00297721]
   Score 0.07289062006617142
MAE is 0.01643771730785943
RMSE is 0.020071659110308695
MSE is 0.00040287149944043797
R^2: 0.07289062006617142


In [19]:
X = train_set[['max_sustained_wind']]
y = train_set['value']

X_test = test_set[['max_sustained_wind']]
y_test = test_set['value']

lrmodel = LinearRegression()
lrmodel.fit(X,y)

y_pred = lrmodel.predict(X)
print(f'Results for linear regression on training data')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Internal Parameters:
   Bias is 0.018979687670944143
   Coefficients: [0.00100935]
   Score 0.024451460468110642
MAE is 0.016810798664489708
RMSE is 0.02058933057287962
MSE is 0.0004239205334393155
R^2: 0.024451460468110642


In [38]:
X = train_set[['stp', 'precipitation' ,'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_poly = poly_process.fit_transform(X)

X_test = test_set[['stp', 'precipitation', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

X_poly_test = poly_process.fit_transform(X_test)

lrmodel = LinearRegression()
lrmodel.fit(X_poly ,y)

y_pred = lrmodel.predict(X_poly)
print(f'Results for linear regression on training data')
print(f'Input: Height, age')
print(f'    Default Settings')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X_poly,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Input: Height, age
    Default Settings
Internal Parameters:
   Bias is -2186.8630810576806
   Coefficients: [ 7.84313153e-04  6.65381693e+00 -6.24835306e+03  1.01258570e+01
 -1.04590959e+01  1.72138863e+01  8.62048529e+00 -2.46142847e+01
 -3.79944008e+00 -6.74726589e-03  1.15488481e+01 -1.71536760e-02
  1.98409890e-02 -3.34847950e-02 -1.69747741e-02  4.78678325e-02
  7.59371637e-03  1.23892042e+02 -3.36572666e+01  1.83845852e+01
 -1.72761441e+01 -1.47218563e+01  7.62994219e+01 -3.07877506e+01
  6.23319988e-02 -1.42253991e-01  5.44391245e-02 -3.36939331e-02
 -8.43738186e-02  1.93407473e-02  5.43820898e-02 -3.50801018e-02
  1.83485700e-02  3.97143512e-02 -6.14179929e-03  1.98876933e-02
  7.69341475e-03 -7.88877291e-02  7.09358488e-03  2.28064139e-03
 -2.15995367e-02  1.37128319e-03  7.48771100e-02 -9.76325664e-03
  5.24917471e-04  2.28046792e-06 -5.30534903e-03  7.04895004e-06
 -9.39795109e-06  1.62790909e-05  8.35733681e-06 -2.32710581e-05

In [39]:
y_test_pred = lrmodel.predict(X_poly_test)
print()
print(f'Results for linear regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
    Polynomial regression with degree 3
MAE is 0.09284859573022523
RMSE is 0.6870589582575579
MSE is 0.47205001212196074
R^2: -3146.664533783589
